In [4]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell
sys.path.append(os.path.join(os.path.dirname(''), '..'))

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

import notebooks.NPA as npa
import notebooks.Physicians as mda

In [7]:
import copy
npa_df = copy.deepcopy(npa.adf)
npa_df

,#,surname,first_name,other_names,cadre,registration_number,registration_date,training_qualification,training_institution,telephone_number,...,is_hard_to_reach,pop_per_health_fac,rural_pop,public_hosp,is_district,poverty_pop,is_city,distance_from_kampala,district_name_all_caps,prop_rural
68,69,Rukundo,Justus,NaN,Anaesthetic Officer,19301,1 November 2012,Post Basic Diploma In Anaesthesia,Mulago Paramedical Training Schools,0757065406,...,0.0,3803.895996,284517.000000,0.0,2.0,0.076,0.0,730.0,RUKUNGIRI,0.971379
84,85,Balayo,James,NaN,Anaesthetic Officer,17434,10 February 2010,Diploma In Anaesthesia,Mulago Paramedical Training Schools,0772311371,...,0.0,1098.000000,0.000000,1.0,2.0,0.037,0.0,0.0,KAMPALA,0.000000
89,90,Ssenteza,Kajubi,Derrick,Anaesthetic Officer,15910,24 January 2011,Diploma In Anaesthesia,Mulago Paramedical Training Schools,0772672900,...,0.0,7660.714355,171805.756348,1.0,2.0,0.073,0.0,242.0,NAKASEKE,0.800959
98,99,Kasuubo,Florence,NaN,Anaesthetic Officer,19304.,6 November 2012,Post Basic Diploma In Anaesthesia,Mulago Paramedical Training Schools,0776571489,...,0.0,6908.108398,244686.515625,0.0,2.0,0.243,0.0,268.0,LUUKA,0.957302
99,100,Kasuubo,Florence,NaN,Anaesthetic Officer,19304.,6 November 2012,Post Basic Diploma In Anaesthesia,Mulago Paramedical Training Schools,0776571489,...,0.0,6908.108398,244686.515625,0.0,2.0,0.243,0.0,268.0,LUUKA,0.957302
111,112,Nakamya,Magoola,Catherine,Anaesthetic Officer,4062,5 December 2012,Diploma In Anaesthesia,Mulago Paramedical Training Schools,0712728740,...,0.0,1098.000000,0.000000,1.0,2.0,0.037,0.0,0.0,KAMPALA,0.000000
190,191,Nakaggwa,Bernadette,NaN,Anaesthetic Officer,19580,14 November 2012,Post Basic Diploma In Anaesthesia,Mulago Paramedical Training Schools,0712835747,...,0.0,1098.000000,0.000000,1.0,2.0,0.037,0.0,0.0,KAMPALA,0.000000
241,242,Kuule,Yason,NaN,Anaesthetic Officer,19200,15 November 2012,Diploma In Anaesthesia,Mulago Paramedical Training Schools,0777887083,...,0.0,1098.000000,0.000000,1.0,2.0,0.037,0.0,0.0,KAMPALA,0.000000
354,355,Benda,Margaret,NaN,Anaesthetic Officer,19432,31 October 2012,Diploma In Anaesthesia,Fortportal School Of Clinical Officers,0773267948,...,0.0,8281.356445,386156.564453,0.0,2.0,0.073,0.0,126.2,LUWERO,0.790333
403,404,Taabu,Geofrey,NaN,Anaesthetic Clinical Officer,19316,15 March 2017,Post Basic Diploma In Anaesthesia,Mulago Paramedical Training Schools,0774515164,...,1.0,5865.714355,194629.738281,0.0,2.0,0.354,0.0,756.0,AMURU,0.948026
